In [1]:
import sys
import time
import json 
import pandas as pd
from app.builtin import *

sys.path.append('../app')  # for let gsdmm to work

Using TensorFlow backend.


In [2]:
models= {
    'ntm': NtmModel,
    'lda': LdaModel,
    'lftm': LftmModel,
    'gsdmm': GsdmmModel
}

# 20 NG Dataset

In [3]:
dataset = '../data/20ng.txt'
labels = '../data/20ng_labels.txt'

In [4]:
with open(labels, "r") as datafile:
    true_topics = [line.rstrip() for line in datafile if line]
true_topics[0:10]

['rec.autos',
 'comp.sys.mac.hardware',
 'comp.sys.mac.hardware',
 'comp.graphics',
 'sci.space',
 'talk.politics.guns',
 'sci.med',
 'comp.sys.ibm.pc.hardware',
 'comp.os.ms-windows.misc',
 'comp.sys.mac.hardware']

In [5]:
results = {}
for name, model in models.items():
    start = time.time()
    m = model()
    print(name)
    results[name] = m.get_corpus_predictions()
    dur = time.time() - start
    print('Done in ' + str(dur))

W0417 13:45:59.506932 4599958976 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


ntm


W0417 13:45:59.693647 4599958976 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0417 13:46:00.028012 4599958976 module_wrapper.py:139] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Done in 1.9874699115753174
lda
Done in 0.5751392841339111
lftm
Done in 0.26327991485595703
gsdmm
Done in 0.38397884368896484


In [6]:
df = pd.DataFrame()
df['true'] = true_topics
for name, model in models.items():
    r = results[name]
    for i in range(0,3):
        key, value = zip(*[x[i] if len(x) > i else (-1, -1) for x in r])
        df['%s_topic_%d'%(name,i)] = key
        df['%s_score_%d'%(name,i)] = value
df

,true,ntm_topic_0,ntm_score_0,ntm_topic_1,ntm_score_1,ntm_topic_2,ntm_score_2,lda_topic_0,lda_score_0,lda_topic_1,...,lftm_topic_1,lftm_score_1,lftm_topic_2,lftm_score_2,gsdmm_topic_0,gsdmm_score_0,gsdmm_topic_1,gsdmm_score_1,gsdmm_topic_2,gsdmm_score_2
0,rec.autos,11,0.227447,4,0.202119,5,0.163517,6,0.845002,12,...,3,0.220588,16,0.073529,1,0.999993,19,6.891301e-06,11,2.122357e-17
1,comp.sys.mac.hardware,14,0.249227,18,0.174287,0,0.165302,5,0.451678,12,...,8,0.146067,17,0.089888,1,1.000000,19,1.552614e-14,4,5.241309e-31
2,comp.sys.mac.hardware,7,0.333553,14,0.272124,5,0.116028,12,0.521352,5,...,10,0.160714,7,0.089286,0,0.000000,1,0.000000e+00,2,0.000000e+00
3,comp.graphics,16,0.254609,0,0.196284,2,0.137768,5,0.602450,12,...,19,0.098592,6,0.084507,1,1.000000,19,7.681433e-09,4,4.168175e-10
4,sci.space,5,0.292701,0,0.186552,10,0.141531,12,0.592231,9,...,1,0.134021,2,0.134021,19,1.000000,4,7.832201e-09,1,6.731073e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11309,sci.med,7,0.601212,8,0.236097,2,0.131402,14,0.730352,12,...,10,0.085714,3,0.080000,0,0.000000,1,0.000000e+00,2,0.000000e+00
11310,comp.sys.mac.hardware,15,0.339457,2,0.160176,19,0.155714,5,0.719951,16,...,17,0.126582,13,0.101266,1,1.000000,4,7.849075e-19,19,1.750207e-19
11311,comp.sys.ibm.pc.hardware,9,0.211234,14,0.181800,12,0.162045,5,0.511405,16,...,8,0.202532,3,0.088608,1,1.000000,19,8.755647e-13,4,6.851880e-28
11312,comp.graphics,4,0.256770,7,0.198837,16,0.172548,2,0.748150,12,...,19,0.092593,2,0.083333,4,1.000000,19,3.562953e-11,1,1.542093e-31


In [7]:
df.index.name = 'id'
df.to_csv('./out/predictions_20ng.csv')

In [8]:
for name, model in models.items():
    m = model()
    with open('./out/'+name+'_topics_20ng.json', 'w') as file:
        json.dump(m.topics(), file)

# AFP dataset

In [3]:
dataset = '../data/afp.txt'
labels = '../data/afp_labels.txt'

In [4]:
with open(labels, "r") as datafile:
    true_topics = [line.rstrip() for line in datafile if line]
true_topics[0:10]

[',,',
 '13000000,13007000 13006000 13008000,',
 '11000000,11014000 11001000 11002000 11006000,',
 '11000000,11014000 11001000 11002000 11006000,',
 '11000000,11014000 11001000 11002000 11006000,',
 '11000000,11014000 11001000 11002000 11006000,',
 '15000000,,',
 '11000000,11014000 11001000 11002000 11006000,',
 '02000000,02003000 02001000 16001000,02003001',
 '15000000,15054000,']

In [5]:
true1, true2, true3 = zip(*[line.split(',') for line in true_topics])
true1[0:10]

('',
 '13000000',
 '11000000',
 '11000000',
 '11000000',
 '11000000',
 '15000000',
 '11000000',
 '02000000',
 '15000000')

In [6]:
results = {}
for name, model in models.items():
    start = time.time()
    m = model()
    print(name)
    results[name] = m.get_corpus_predictions()
    dur = time.time() - start
    print('Done in ' + str(dur))

W0417 13:45:03.727105 4663692736 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


ntm


W0417 13:45:03.899070 4663692736 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0417 13:45:04.289768 4663692736 module_wrapper.py:139] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

/Users/pasquale/git/ted-talk-topic-extraction/app/builtin/doc2topic/models.py:18: RuntimeWarning: invalid value encountered in true_divide
  L1normalize = (lambda x: x/L1(x))


Done in 5.400324106216431
lda
Done in 4.530890941619873
lftm
Done in 3.830026865005493
gsdmm
Done in 3.0285561084747314


In [7]:
df = pd.DataFrame()
df['true'] = true1
df['true2'] = true2
df['true3'] = true3
for name, model in models.items():
    r = results[name]
    for i in range(0,3):
        key, value = zip(*[x[i] if len(x) > i else (-1, -1) for x in r])
        df['%s_topic_%d'%(name,i)] = key
        df['%s_score_%d'%(name,i)] = value
df

,true,true2,true3,ntm_topic_0,ntm_score_0,ntm_topic_1,ntm_score_1,ntm_topic_2,ntm_score_2,lda_topic_0,...,lftm_topic_1,lftm_score_1,lftm_topic_2,lftm_score_2,gsdmm_topic_0,gsdmm_score_0,gsdmm_topic_1,gsdmm_score_1,gsdmm_topic_2,gsdmm_score_2
0,,,,9,0.342370,1,0.323482,16,0.244672,0,...,1,0.006803,2,0.006803,15,0.999825,16,0.000175,5,2.455619e-09
1,13000000,13007000 13006000 13008000,,5,0.296621,9,0.129710,1,0.121188,12,...,7,0.251570,3,0.055781,0,0.000000,1,0.000000,2,0.000000e+00
2,11000000,11014000 11001000 11002000 11006000,,13,0.694453,7,0.292373,0,0.007895,5,...,15,0.179487,2,0.094017,10,0.999995,16,0.000005,5,1.009442e-12
3,11000000,11014000 11001000 11002000 11006000,,13,0.420780,7,0.325828,8,0.231620,14,...,3,0.196262,15,0.196262,10,0.999999,16,0.000001,15,2.206482e-11
4,11000000,11014000 11001000 11002000 11006000,,7,0.413961,13,0.412301,16,0.135707,12,...,15,0.165354,10,0.086614,10,0.999998,16,0.000002,15,7.401703e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125511,16000000,16002000 16001000,,9,0.410465,2,0.200190,11,0.109299,14,...,12,0.093545,3,0.055575,0,0.000000,1,0.000000,2,0.000000e+00
125512,01000000 08000000,01016000 08006000 01005000 01007000 01026000,08003002 08003003,10,0.280207,9,0.143741,11,0.138465,12,...,8,0.048621,3,0.045778,0,0.000000,1,0.000000,2,0.000000e+00
125513,01000000 02000000,01026000,,12,0.228517,8,0.194645,2,0.189475,13,...,5,0.071784,9,0.036247,0,0.000000,1,0.000000,2,0.000000e+00
125514,14000000 08000000 11000000,14023000 08003000 12006000,,13,0.275202,3,0.211507,12,0.205389,0,...,12,0.250592,10,0.169428,0,0.000000,1,0.000000,2,0.000000e+00


In [8]:
df.index.name = 'id'
df.to_csv('./out/predictions_afp.csv')

In [9]:
for name, model in models.items():
    m = model()
    with open('./out/'+name+'_topics_afp.json', 'w') as file:
        json.dump(m.topics(), file)